In [1]:
def yolo_to_corners(label, img_width, img_height):
    """
    Преобразует лейбл YOLO в координаты углов прямоугольника.

    :param label: Лейбл YOLO в формате [class_id, x_center, y_center, width, height]
    :param img_width: Ширина изображения в пикселях
    :param img_height: Высота изображения в пикселях
    :return: Кортеж с координатами углов прямоугольника (x_min, y_min, x_max, y_max)
    """
    class_id, x_center_norm, y_center_norm, width_norm, height_norm = label

    # Денормализация координат+

    x_center = x_center_norm * img_width
    y_center = y_center_norm * img_height
    width = width_norm * img_width
    height = height_norm * img_height

    # Вычисление координат углов
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return x_min, y_min, x_max, y_max

In [2]:
import os 
import cv2 as cv
import shutil

In [3]:
path_img_train = "F:/datasets/images/train/"
path_label_train = "F:/datasets/labels/train/"

path_img_test = "F:/datasets/images/val/"
path_label_test = "F:/datasets/labels/val/"

files_img = sorted(os.listdir(path_img_train))
files_label = sorted(os.listdir(path_img_train))

In [ ]:
for i, filename in enumerate(files_img):
    img = cv.imread(path_img_train + filename)
    h, w = img.shape[:2]
    label_list = []

    with open(path_label_train + filename.replace(".jpg", ".txt"), 'r') as f:
        for line in f.readlines():
            label, x1, y1, x2, y2 = line.split()
            label_list.append([int(label), float(x1), float(y1), float(x2), float(y2)])

    
    for label in label_list:
        x_min, y_min, x_max, y_max = yolo_to_corners(label, w, h)
        print(x_min, y_min, x_max, y_max)
        cv.rectangle(img, (x_min, y_min),(x_max, y_max), (255,0,0),3)

    cv.imshow("image", img)


    while True:
        key = cv.waitKey(0) & 0xFF
        if key == ord('e'):
            print(f"Продолжение цикла. {i}")
            shutil.move(path_img_train + filename, "F:/datasets/images_corrent/train/" + filename)
            shutil.move(path_label_train + filename.replace(".jpg", ".txt"), "F:/datasets/labels_corrent/train/" + filename.replace(".jpg", ".txt"))
            break
        elif key == ord('q'):  
            print(f"Неправильная разметка. {i}")
            shutil.move(path_img_train + filename, "F:/datasets/images_incorrent/train/" + filename)
            shutil.move(path_label_train + filename.replace(".jpg", ".txt"), "F:/datasets/labels_incorrent/train/" + filename.replace(".jpg", ".txt"))
            break

cv.destroyAllWindows()

605 107 716 262
Продолжение цикла. 0
151 232 208 319
277 115 499 407
518 258 559 330
670 272 721 339
Продолжение цикла. 1
199 86 461 410
Продолжение цикла. 2
567 31 635 126
Продолжение цикла. 3
62 384 78 424
111 398 134 426
514 403 565 473
631 429 685 504
695 359 713 381
824 451 857 521
Продолжение цикла. 4
261 287 616 823
Продолжение цикла. 5
141 53 184 94
144 323 238 435
434 177 538 309
835 378 921 537
Продолжение цикла. 6
624 123 842 397
Продолжение цикла. 7
329 436 366 483
Продолжение цикла. 8
127 200 208 280
466 206 550 334
609 72 745 300
Продолжение цикла. 9
230 418 280 552
508 283 626 429
Продолжение цикла. 10
535 281 646 434
930 0 1001 140
Продолжение цикла. 11
408 82 567 272
Продолжение цикла. 12
373 129 430 192
691 171 749 230
896 126 961 205
964 120 1019 238
997 139 1023 272
Продолжение цикла. 13
445 333 624 535
Продолжение цикла. 14
339 117 453 305
629 132 739 312
Продолжение цикла. 15
209 207 341 383
525 105 685 319
845 177 909 291
869 361 1022 629
Продолжение цикла. 16
16

In [4]:
ds = 'F:/datasets/'
ln_1="#Paths \n"
ln_2='train: ' +"'"+ path_img_train +"' \n"
ln_3='val: ' +"'" + path_img_test +"' \n"
ln_4='test: ' +"'" + path_img_test +"' \n"
ln_5="\n"
ln_6='# Classes\n'
ln_7='names:\n'
ln_8='  0: face'
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8]

with open("config.yaml", 'w') as f:
    f.writelines(config_lines)

In [5]:
from ultralytics import YOLO

In [6]:
model = YOLO('yolo11s.yaml').load('yolo11s.pt')

Transferred 499/499 items from pretrained weights


In [7]:
model.train(
    data='F:\datasets\config.yaml',
    epochs=50,
    batch=16,
    name='model_1',
    device=0,
    save=True           
)

Ultralytics 8.3.71  Python-3.11.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.yaml, data=F:\datasets\config.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=model_115, exist_ok=False, pretrained=yolo11s.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning F:\datasets\labels\train.cache... 13190 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13190/13190 [00:00<?, ?it/s]
val: Scanning F:\datasets\labels\val.cache... 3347 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3347/3347 [00:00<?, ?it/s]


Plotting labels to runs\detect\model_115\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\model_115
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      4.69G      1.323      1.094      1.199         37        640: 100%|██████████| 825/825 [04:00<00:00,  3.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [00:31<00:00,  3.31it/s]

                   all       3347      10299      0.877      0.757      0.844      0.543



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/50      4.94G      1.239     0.7569      1.091         57        640: 100%|██████████| 825/825 [03:52<00:00,  3.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [00:30<00:00,  3.46it/s]


                   all       3347      10299       0.86      0.731      0.817      0.507

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/50      4.79G      1.278     0.8154      1.113        102        640:  47%|████▋     | 384/825 [01:44<01:59,  3.69it/s]


KeyboardInterrupt: 